In [2]:
import pyodbc
import pandas as pd

In [3]:
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=ONLINETUTOR;"   # or your actual server name
    "DATABASE=Energy Project;"
    "Trusted_Connection=yes;"
)

In [4]:
query = "SELECT * FROM CityMonthlyEnergyWeatherView"
df = pd.read_sql(query, conn)

C:\Users\Raki_\AppData\Local\Temp\ipykernel_21532\1262125030.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
print(df)

         City    Month  Avg_Electricity_kWh  Avg_Gas_Usage  Avg_Temp_C  \
0     Calgary  2022-01              5725.15        1064.38         9.9   
1     Calgary  2022-02              5859.08        1087.66         4.7   
2     Calgary  2022-03              4432.64         515.81        12.4   
3     Calgary  2022-04              4801.43         550.08         8.6   
4     Calgary  2022-05              4746.62         722.07       -13.0   
..        ...      ...                  ...            ...         ...   
163  Winnipeg  2023-08              7387.80         291.73        -3.7   
164  Winnipeg  2023-09              4862.07         562.91         2.9   
165  Winnipeg  2023-10              4137.13         509.28        -6.2   
166  Winnipeg  2023-11              4532.64         595.92        19.0   
167  Winnipeg  2023-12              5846.54        1103.97       -10.9   

     Total_Heating_Days  Total_Cooling_Days  
0                   243                   0  
1                  

In [6]:
df.isnull().sum()

City                   0
Month                  0
Avg_Electricity_kWh    3
Avg_Gas_Usage          3
Avg_Temp_C             0
Total_Heating_Days     0
Total_Cooling_Days     0
dtype: int64

In [7]:
df_cleaned = df.dropna(subset=['Avg_Electricity_kWh','Avg_Gas_Usage'])

In [8]:
print("After Cleaned:",df_cleaned.shape)

After Cleaned: (164, 7)


In [9]:
df_cleaned.duplicated().sum()

np.int64(0)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [11]:
x = df_cleaned[['Avg_Temp_C', 'Total_Heating_Days', 'Total_Cooling_Days']]
y = df_cleaned['Avg_Electricity_kWh']


In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [13]:
mo = LinearRegression()
mo.fit(x_train,y_train)

LinearRegression()

In [14]:
predict_Y = mo.predict(x_test)

In [15]:
print(predict_Y)

[5695.14096064 5719.2050229  5696.92633435 5499.99760829 5392.28541082
 5659.68395824 5775.54045268 5492.43885759 5745.37909372 5303.47009011
 5602.04074273 5552.44513409 5644.564245   5387.56119164 5386.6163478
 5295.91133941 5735.75821258 5621.91155089 5374.33337791 5563.30214539
 5593.0734191  5567.56263549 5735.28252442 5602.52185747 5423.46525746
 5603.9304304  5730.94012165 5413.07197525 5727.27768745 5464.09354247
 5681.30431438 5593.0734191  5409.2925999 ]


In [16]:
print(y_test.values)


[4847.55       7269.11666667 5939.545      7671.28       7254.38
 7183.45       4335.34       4793.2        7927.93333333 6251.44
 7181.46       6217.84       4295.94       8059.4        6274.87
 7017.59       5570.61       4633.16666667 3906.63       4185.73
 4443.82       7239.86       5575.63       4465.2        7102.64
 4563.82       4523.28666667 4286.66       2817.52       4862.07
 6828.12       7321.49       5590.73      ]


In [17]:
mse=mean_squared_error(y_test,predict_Y)

In [18]:
print("Mean Squared Error:", mse)

Mean Squared Error: 2055847.4865051445


In [19]:
import numpy as np
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1433.8226830766573


In [20]:
print("Max:", y_test.max())
print("Min:", y_test.min())
print("Average:", y_test.mean())

Max: 8059.4
Min: 2817.52
Average: 5770.827828282829


In [21]:
percent_error = (rmse / y_test.mean()) * 100
print("Percent Error:", percent_error)

Percent Error: 24.846048534830516


In [22]:
import joblib
joblib.dump(mo, 'electricity_model.pkl')

['electricity_model.pkl']